In [1]:
from radiopy.radiation import u_lum_spec_en
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from physipy import *
from physipy.quantity.utils import *
from physipy.quantity.calculus import ndvectorize
mm = units["mm"]
mum = units["mum"]


setup_matplotlib()
ln = np.log

# Basic units
J = units["J"]
W = units["W"]
nm = units["nm"]
mum = units["mum"]
Hz = units["Hz"]

# Basic constants
c = scipy_constants["c"]
hp = scipy_constants["h"]
kB = scipy_constants["k"]
pi = np.pi


# Custom unit
ph = Quantity(1, Dimension(None), symbol="ph")

# Derived usefull units
u_lum_en = W * m**-2 * sr**-1
u_lum_ph = u_lum_en * ph * s**-1 * W**-1
u_lum_spec_en = u_lum_en / m
u_lum_spec_ph = u_lum_ph / m
u_lum_freq_en = u_lum_en / Hz
u_lum_freq_ph = u_lum_ph / Hz
u_exi_en = u_lum_en * sr
u_exi_ph = u_lum_ph * sr

# Constants
aperys_constant = 1.2020569031595942853997381615114499907649862923405
a_2 = 1.59362426004
a_3 = 2.82143937212
a_4 = 3.9206903948
a_5 = 4.965114231744276
lambert_sol = a_5

In [2]:
from radiopy.radiation import planck_spec_en, wien_displacement_spec_en, lum_max_spec_en, lum_spec_en_bande, planck_spec_en_

# Interactive Ax

In [3]:
%matplotlib ipympl

In [4]:
import matplotlib.pyplot as plt
import ipywidgets as ipyw
from physipy.qwidgets.qipywidgets import *


class IAx(ipyw.HBox):
    xmin = traitlets.Float(allow_none=False)
    xmax = traitlets.Float(allow_none=False)
    ymin = traitlets.Float(allow_none=False)
    ymax = traitlets.Float(allow_none=False)
    xmax_smin = traitlets.Float(allow_none=False)
    xmax_smax = traitlets.Float(allow_none=False)
    xmin_smin = traitlets.Float(allow_none=False)
    xmin_smax = traitlets.Float(allow_none=False)
    
    def __init__(self, xfavunit=None, yfavunit=None):
        
        super().__init__()
        
        out_w = ipyw.Output()
        with out_w:
            fig, ax = plt.subplots()
        self.fig = fig
        self.ax = ax
        
        # create widgets
        xmin, xmax = self.ax.get_xlim()
        ymin, ymax = self.ax.get_ylim()
        self.xmin = xmin
        self.xmax = xmax
        self.ymin = ymin
        self.ymax = ymax
        self.xmin_w = ipyw.FloatSlider(self.xmin, min=self.xmin, max=self.xmax, description="xmin")
        self.xmax_w = ipyw.FloatSlider(self.xmax, min=self.xmin, max=self.xmax, description="xmax")
        self.ymin_w = ipyw.FloatSlider(self.ymin, min=self.ymin, max=self.ymax, description="ymin")
        self.ymax_w = ipyw.FloatSlider(self.ymax, min=self.ymin, max=self.ymax, description="ymax")
        title_w = ipyw.Text(description="title")
        xlabel_w = ipyw.Text(description="xlabel")
        ylabel_w = ipyw.Text(description="ylabel")
        grid_enable_w = ipyw.Checkbox(value=False, description="Grid")
        grid_which_w = ipyw.Dropdown(description="which", options=["both", "major", "minor"])
        grid_axis_w = ipyw.Dropdown(description="axis", options=["both", "x", "y"])
        grid_w = ipyw.VBox([grid_enable_w, grid_which_w, grid_axis_w])
        legend_w = ipyw.Button(description="Legend")
        self.lines_w = ipyw.Label()
        self.list_of_lines = []
        
        
        # link widgets
        self.xmin_w.observe(self._update_xmin,                        "value")
        self.xmax_w.observe(self._update_xmax,                        "value")
        self.ymin_w.observe(self._update_ymin,                        "value")
        self.ymax_w.observe(self._update_ymax,                        "value")
        title_w.observe(self._update_title,                      "value")
        xlabel_w.observe(self._update_xlabel,                    "value")
        ylabel_w.observe(self._update_ylabel,                    "value")
        grid_enable_w.observe(self._update_grid_enable,          "value")
        grid_which_w.observe(self._update_grid_which,            "value")
        grid_axis_w.observe(self._update_grid_axis,              "value")
        legend_w.on_click(self._update_legend)
        
        xmin_link = traitlets.link((self.xmin_w, "value"), (self, "xmin"))
        xmax_link = traitlets.link((self.xmax_w, "value"), (self, "xmax"))
        ymin_link = traitlets.link((self.ymin_w, "value"), (self, "ymin"))
        ymax_link = traitlets.link((self.ymax_w, "value"), (self, "ymax"))
        xmin_xmax_min_link = traitlets.link((self.xmin_w, 'min'), (self.xmax_w, 'min'))
        xmin_xmax_max_link = traitlets.link((self.xmin_w, 'max'), (self.xmax_w, 'max'))
        ymin_ymax_min_link = traitlets.link((self.ymin_w, 'min'), (self.ymax_w, 'min'))
        ymin_ymax_max_link = traitlets.link((self.ymin_w, 'max'), (self.ymax_w, 'max'))

        xmin_smin = traitlets.link((self.xmin_w, "min"),   (self, "xmin_smin"))
        xmin_smax = traitlets.link((self.xmin_w, "max"),   (self, "xmin_smax"))
        xmax_smin = traitlets.link((self.xmax_w, "min"),   (self, "xmax_smin"))
        xmax_smax = traitlets.link((self.xmax_w, "max"),   (self, "xmax_smax"))
                                   
        # box widgets
        self.vbox = ipyw.VBox([
            self.xmin_w,
            self.xmax_w,
            self.ymin_w,
            self.ymax_w,
            title_w,
            xlabel_w,
            ylabel_w,
            grid_w,
            self.lines_w,
            legend_w,
        ])
        self.children = [
            out_w,
            self.vbox
        ]


    def _update_xmin(self, change):
        #_, self.xmax = self.ax.get_xlim()
        self.xmin = change["new"]
        self.ax.set(xlim=(self.xmin, self.xmax))
    
    def _update_xmax(self, change):
        #self.xmin, _ = self.ax.get_xlim()
        self.xmax = change["new"]
        self.ax.set(xlim=(self.xmin, self.xmax))
    
    def _update_ymin(self, change):
        #ymin, _ = self.ax.get_ylim()
        self.ymin = change["new"]
        self.ax.set(ylim=(self.ymin, self.ymax))
    
    def _update_ymax(self, change):
        #xmin, xmax = self.ax.get_ylim()        
        self.ymax = change["new"]
        self.ax.set(ylim=(self.ymin, self.ymax))
    
    def _update_title(self, change):
        self.ax.set(title=change["new"])
    
    def _update_xlabel(self, change):
        self.ax.set(xlabel=change["new"])
    
    def _update_ylabel(self, change):
        self.ax.set(ylabel=change["new"])
    
    def _update_grid_enable(self, change):
        self.ax.grid(b=change["new"])

    def _update_grid_which(self, change):
        self.ax.grid(which=change["new"])

    def _update_grid_axis(self, change):
        self.ax.grid(axis=change["new"])
        
    def _update_legend(self, b):
        """Not working after line was removed"""
        try:
            self.ax.legend()
        except:
            self.ax.get_legend().remove()

    def add_line(self, func, xmin, xmax, xfavunit=None, label=""):
        
        ech_x = np.linspace(xmin, xmax)
        
        
        xmin = quantify(xmin)
        xmax = quantify(xmax)

        
        
        # TODO : update x-sliders
        # store a list of xmins and xmaxs
        # store a list of lines
        # if no xmin/xmax, use the current min/max
        
        # remove lines : ax.ax.lines[0].remove()
        # update legend when removing
        # use self.favunit
        
        
        if hasattr(func, "name"):
            label = func.name
        # compute ys
        try:
            ech_y = func(ech_x)
        except: 
            ech_y = physipy.quantity.utils.asqarray([func(x) for x in ech_x])
            
        # plot
        if xfavunit is None:
            lines = self.ax.plot(ech_x, ech_y, label=label)
        else:
            ech_x.favunit = xfavunit
            lines = self.ax.plot(ech_x, ech_y, label=label)

        self.ax.relim()
        self.ax.autoscale()
        
        xmin, xmax = self.ax.get_xlim()
        ymin, ymax = self.ax.get_ylim()
        # update xmin/xmax for ax :
        if xmin < self.xmin_w.min:
            self.xmin_w.min = xmin
            self.xmin = xmin
        if ymin < self.ymin_w.min:
            self.ymin_w.min = ymin
            self.ymin = ymin
        if xmax > self.xmax_w.max:
            self.xmax_w.max = xmax
            self.xmax = xmax
        if ymax > self.ymax_w.max:
            self.ymax_w.max = ymax
            self.ymax = ymax
        
        #self.xmin = xmin
        #self.xmax = xmax
         
            
            
        self.list_of_lines.append(label)
        self.lines_w.value = str(self.list_of_lines)
        
    def add_iline(self, func, xmin, xmax, params, label=""):
        # create widgets to controls additional args
        
        pass
    
            
    def remove_line(self, n, label=None):
        self.ax.lines[n].remove()
        self.list_of_lines.pop(n)
        self.lines_w.value = str(self.list_of_lines)
        self.ax.legend()

In [5]:
ax = IAx()
ax

# avoir une liste de bande spectrale pour comparaison
# bouton pour update legend
# dropdown pour xfavunit et yvafunit

IAx(children=(Output(), VBox(children=(FloatSlider(value=0.0, description='xmin', max=1.0), FloatSlider(value=…

In [10]:
ax.add_line(lambda l: planck_spec_en(l, 250*K), 2*mum, 12*mum, xfavunit=mum, label="planck(250K)")

In [7]:
ax.add_line(lambda l: planck_spec_en(l, 200*K), 2*mum, 20*mum, xfavunit=mum, label="planck(200K)")


In [9]:
print(ax.list_of_lines)

['planck(200K)']


In [8]:
ax.remove_line(0)

### use ndvectorize and lambda function
 - dirty
 - lose favunit

In [3]:
%matplotlib ipympl

In [65]:
import ipywidgets as ipyw
T = 300*K
T_min = 250*K
T_max = 350*K

planck = ndvectorize(lambda lmbda: planck_spec_en(lmbda, T))

ech_x_lmbda = np.linspace(0.2, 15, 100) * mum
ech_x_lmbda.favunit = mum
ech_y = planck(ech_x_lmbda)

ech_T = np.linspace(T_min, T_max, 100)
wien_lmbda = ndvectorize(lambda T:wien_displacement_spec_en(T))(ech_T)


lmbda_min_int = 4*mum
lmbda_max_int = 6*mum
int_init = lum_spec_en_bande(lmbda_min_int, lmbda_max_int, T)



In [66]:
lum_max = ndvectorize(lambda T:lum_max_spec_en(T))(ech_T)

In [67]:
out_w = ipyw.Output()

with out_w:    
    fig, ax = plt.subplots()
    ax.plot(ech_x_lmbda, ech_y, label="$"+planck_spec_en.latex+"$")
    ax.plot(wien_lmbda, lum_max, label="lum max")
    
    # Make the shaded region
    from matplotlib.patches import Polygon
    ix = np.linspace(lmbda_min_int, lmbda_max_int)
    iy = planck_spec_en_(ix, T)
    verts = [(lmbda_min_int.value*1e6, 0), *zip(ix.value*1e6, iy.value), (lmbda_max_int.value*1e6, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)
    print(poly)
    plt.legend()

    
xmin, xmax = ax.get_xlim()
ymin, ymax = ax.get_ylim()

xechmin = min(ech_x_lmbda).value*1e6
xechmax = max(ech_x_lmbda).value*1e6

xechmin_w = ipyw.FloatSlider(xechmin, min=xechmin, max=xechmax, description="xechmin")
xechmax_w = ipyw.FloatSlider(xechmax, min=xechmin, max=xechmax, description="xechmax")

xmin_w = ipyw.FloatSlider(xmin, min=xmin, max=xmax, description="xmin")
xmax_w = ipyw.FloatSlider(xmax, min=xmin, max=xmax, description="xmax")
ymin_w = ipyw.FloatSlider(ymin, min=ymin, max=ymax, description="ymin")
ymax_w = ipyw.FloatSlider(ymax, min=ymin, max=ymax, description="ymax")
title_w = ipyw.Text(description="title")
xlabel_w = ipyw.Text(description="xlabel")
ylabel_w = ipyw.Text(description="ylabel")
line_legend_w = ipyw.Text(description="line legend")
temp_w = ipyw.FloatSlider(300, min=T_min.value, max=T_max.value, description="Temperature(K)")
nech_w = ipyw.IntSlider(min=10, max=500, description="n")

#with out_w:
#    fig, ax = plt.subplots(figsize=(6, 4))
#    lines = ax.plot(ech_x_lmbda, ech_y, "o")
#    ax.set(
#        ylabel='Temperature (deg C)',
#        xlabel='$\lambda$',
#        title='Planck',
#    )

box = ipyw.VBox([
    xmin_w,
    xmax_w,
    ymin_w,
    ymax_w,
    title_w,
    xlabel_w,
    ylabel_w,
    line_legend_w,
    temp_w,
    xechmin_w,
    xechmax_w,
    nech_w,
])


def update_xmin(change):
    xmin, xmax = ax.get_xlim()
    ax.set(xlim=(change["new"], xmax))


def update_xmax(change):
    xmin, xmax = ax.get_xlim()
    ax.set(xlim=(xmin, change["new"]))


def update_ymin(change):
    ymin, ymax = ax.get_ylim()
    ax.set(ylim=(change["new"], ymax))


def update_ymax(change):
    xmin, xmax = ax.get_ylim()
    ax.set(ylim=(ymin, change["new"]))


def update_title(change):
    ax.set(title=change["new"])


def update_xlabel(change):
    ax.set(xlabel=change["new"])


def update_ylabel(change):
    ax.set(ylabel=change["new"])


def update_legend(change):
    if change["new"] == "":
        ax.get_legend().remove()
    else:
        ax.legend([change["new"]])


def update_temp(change):
    func = ndvectorize(lambda lmbda: planck_spec_en(lmbda, change["new"]*K))
    ech_y = func(ech_x_lmbda)
    ech_y.favunit = u_lum_spec_en
    ax.lines[0].set_data(ech_x_lmbda, ech_y)
    ax.relim()
    ax.autoscale_view()


def update_echxmin(change):
    temp = temp_w.value
    xmaxech = xechmax_w.value
    nech = nech_w.value
    func = ndvectorize(lambda lmbda: planck_spec_en(lmbda, temp*K))
    ech_x_lmbda = np.linspace(change["new"], xmaxech, nech)*mum
    ech_y = func(ech_x_lmbda)
    ax.lines[0].set_data(ech_x_lmbda, ech_y)
    ax.relim()
    ax.autoscale_view()

    
def update_echxmax(change):
    temp = temp_w.value
    xminech = xechmin_w.value
    nech = nech_w.value
    func = ndvectorize(lambda lmbda: planck_spec_en(lmbda, temp*K))
    ech_x_lmbda = np.linspace(xminech, change["new"], nech)*mum
    ech_y = func(ech_x_lmbda)
    ax.lines[0].set_data(ech_x_lmbda, ech_y)
    ax.relim()
    ax.autoscale_view()

    
def update_nech(change):
    temp = temp_w.value
    xminech = xechmin_w.value
    xmaxech = xechmax_w.value
    nech = nech_w.value
    func = ndvectorize(lambda lmbda: planck_spec_en(lmbda, temp*K))
    ech_x_lmbda = np.linspace(xminech, xmaxech, change["new"])*mum
    ech_y = func(ech_x_lmbda)
    ax.lines[0].set_data(ech_x_lmbda, ech_y)
    ax.relim()
    ax.autoscale_view()

In [68]:
temps = np.linspace(250, 350, 11)*K
print(temps)
x_temps = [wien_displacement_spec_en(T) for T in temps]
y_temps = [lum_max_spec_en(T) for T in temps]

[250. 260. 270. 280. 290. 300. 310. 320. 330. 340. 350.] K


In [69]:
with out_w:
    ax.grid(True, which="both", axis="both")
    for i, txt in enumerate(temps):
        ax.annotate(txt, (x_temps[i], y_temps[i]))
        

In [70]:
    
xmin_w.observe(update_xmin,          "value")
xmax_w.observe(update_xmax,          "value")
ymin_w.observe(update_ymin,          "value")
ymax_w.observe(update_ymax,          "value")
title_w.observe(update_title,        "value")
xlabel_w.observe(update_xlabel,      "value")
ylabel_w.observe(update_ylabel,      "value")
line_legend_w.observe(update_legend, "value")
temp_w.observe(update_temp,          "value")
xechmin_w.observe(update_echxmin,    "value")
xechmax_w.observe(update_echxmax,    "value")
nech_w.observe(update_nech,          "value")


display(ipyw.HBox([out_w, box]))